# How to play videos

There is no simple way to play video in Python especially on the Raspberry Pi with its limited power and resources. 

The most performant video player on Raspberry Pi is omxplayer, which is run from the terminal. It is usually preinstalled on Raspbian. If not, install with
```
> sudo apt-get install omxplayer
```
then play a video with
```
> omxplayer video_file_path
```
The Python module *omxplayer-wrapper* exists to control omxplayer from Python, but it did not work for me out of the box. 

The idea now is to run and kill omxplayer instances to play video and overlay the display with a black surface when no video is playing using the *pygame* module.

If you encounter errors when installing *pygame* with *pip*
```
> pip3 install pygame
```

use the package provided by Raspbian:
```
> sudo apt-get install python3-pygame
```

In [51]:
import os
import sys
import subprocess
import pygame
import time

Open a pygame display and fill it black. Note that we use a 640x480 window here in the tutorial, so you still have access to this notebook. For your work you will usually do everything fullscreen (see the line that includes *pygame.FULLSCREEN*).

In [54]:
# In this tutorial we show only a 640x480 window. In real life you would have
# pygame's black display fill the whole screen and run omxplayer in fullscreen.

# Set window position and size variables (only for this tutorial)
winpos = (100,100)
winsize = (640, 480)

# set window position (only for this tutorial)
os.environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % winpos

# init pygame
pygame.init()
pygame.mouse.set_visible(False)

# use pygame to show black screen when no video is shown.
# You want to use the FULLSCREEN line in your work.

screen = pygame.display.set_mode(winsize) # only for this tutorial
# screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)

# fill display black
screen.fill((0, 0, 0))

<rect(0, 0, 640, 480)>

Now you can play a video using the system's omxplayer. Again, we start omxplayer in a window for you to have access to the notebook. Look at the man-page of omxplayer to get information about the options used as well as further options:
```
> man omxplayer
```
Note that the execution of omxplayer is *blocking*, which means that your program waits for omxplayer to finish! This may be useful if you want to run some code after the video has played, but you loose control while the video is playing.

In [55]:
os.system('omxplayer --no-osd --no-keys --win 100,100,740,580 resources/vid.mp4')

0

An option with more control over the omxplayer process is to use *subprocess*, which starts a program in a new process. Note that the call is *non-blocking*, which means that the execution of your program does not wait at this line of code. Now you can run some other code while the video plays, e.g. kill all omxplayer instances.

With the object returned by subprocess.Popen you can wait for the process to end or test if the process has terminated.

Note that Popen requires a list of arguments as input, not a string.

In [61]:
# run omxplayer in a separate process
cmd = ['omxplayer', '--no-osd', '--no-keys', '--win' ,'100,100,740,580', 'resources/vid.mp4']
p = subprocess.Popen(cmd)

In [49]:
# run omxplayer in a separate process and wait for its termination
cmd = ['omxplayer', '--no-osd', '--no-keys', '--win' ,'100,100,740,580', 'resources/vid.mp4']
p = subprocess.Popen(cmd)

print('Waiting for omxplayer to terminate.')

p.wait() # wait blocks the execution of your code until the process has terminated

print('omxplayer has terminated!')

Waiting for omxplayer to terminate.
omxplayer has terminated!


In [78]:
# run omxplayer in a separate process and repeatedly check for its termination
starttime = time.time()

cmd = ['omxplayer', '--no-osd', '--no-keys', '--win' ,'100,100,740,580', 'resources/vid.mp4']
p = subprocess.Popen(cmd)

print('omxplayer started')

while p.poll() is None: # poll return None as long as the process is running
    time.sleep(0.5)
    print('omxplayer is running now for {} seconds.'.format(time.time() - starttime))
    
print('omxplayer has terminated!')

omxplayer started
omxplayer is running now for 0.5264863967895508 seconds.
omxplayer is running now for 1.0286943912506104 seconds.
omxplayer is running now for 1.5297937393188477 seconds.
omxplayer is running now for 2.0311126708984375 seconds.
omxplayer is running now for 2.5317862033843994 seconds.
omxplayer is running now for 3.0331056118011475 seconds.
omxplayer is running now for 3.5364232063293457 seconds.
omxplayer is running now for 4.038497447967529 seconds.
omxplayer is running now for 4.541908025741577 seconds.
omxplayer has terminated!


In [79]:
# kill all omxplayer instances
os.system('killall omxplayer.bin')

256

In [84]:
# quit pygame
pygame.quit()

#### Running in a loop and exiting pygame

Pygame provides you events when something happens, e.g. mouse or keyboard input or when the pygame display is about to close. Here is a full program running a loop to check for the QUIT event, ESQ and Q key press to quit and play a video on a button press on pin 11. Again, the button is connected to pin 11 on one side and to GND on the other side (pull-up resistor scenario).

In [86]:
import os
import sys
import subprocess
import pygame
import time
import RPi.GPIO as GPIO

GPIO.setmode(GPIO.BOARD)
GPIO.setup(11, GPIO.IN, pull_up_down=GPIO.PUD_UP)

# Set window position and size variables (only for this tutorial)
winpos = (100,100)
winsize = (640, 480)

# set window position (only for this tutorial)
os.environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % winpos

# init pygame
pygame.init()
pygame.mouse.set_visible(False)

# use pygame to show black screen when no video is shown.
# You want to use the FULLSCREEN line in your work.
screen = pygame.display.set_mode(winsize) # only for this tutorial
# screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)

# fill display black
screen.fill((0, 0, 0))


lastPinValue = GPIO.HIGH

quit = False
runtime = 30.0
starttime = time.time()

while time.time() - starttime < runtime and quit == False:
    
    # if QUIT event occurs or ESC or Q button is pressed, 
    # set quit variable to True to exit the loop
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit = True   
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE or event.unicode == 'q':
                quit = True
       
    # play video when button is pressed
    if GPIO.input(11) == GPIO.LOW and lastPinValue == GPIO.HIGH:
        cmd = ['omxplayer', '--no-osd', '--no-keys', '--win' ,'100,100,740,580', 'resources/vid.mp4']
        p = subprocess.Popen(cmd)
        lastPinValue = GPIO.LOW
    elif GPIO.input(11) == GPIO.HIGH and lastPinValue == GPIO.LOW:
        # uncomment the line below to play video only as long as button is pressed
        # os.system('killall omxplayer.bin')  
        lastPinValue = GPIO.HIGH
        
# before application exits...
os.system('killall omxplayer.bin') # kill omxplayer if running
pygame.quit()                      # quit pygame
GPIO.cleanup()                     # free GPIO resources